## Install Sentinel-Hub

In [ ]:
pip install sentinelhub --upgrade

     |████████████████████████████████| 194kB 6.7MB/s 
     |████████████████████████████████| 133kB 6.7MB/s 
     |████████████████████████████████| 7.2MB 9.6MB/s 
     |████████████████████████████████| 6.5MB 34.0MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
  Created wheel for sentinelhub: filename=sentinelhub-3.1.0-cp36-none-any.whl size=196404 sha256=78ee1a86eb04f5464ab854181000e59abfdbd6f7bcc88b0126d5604feccf6fdb
  Stored in directory: /root/.cache/pip/wheels/90/6c/df/be1525ead52f5c2ce49f0880e73587723bf25caac7295f9db2
  Created wheel for utm: filename=utm-0.7.0-cp36-none-any.whl size=6094 sha256=f73848c720b25b381ec0e683d0fb363e21f9b40dca69f86afeaae510809ddc7e
  Stored in directory: /root/.cache/pip/wheels/d3/a0/c4/93c7535fe705e70d2376505fd2683f91524a50a944a15212d9
Successfully built sentinelhub utm
ERROR: botocore 1.19.57 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


### Put the path to the folder where we want to save the files from Sentinelhub

In [ ]:
rootPath="/content/gdrive/My Drive/WORK/C&T/SENTINEL-HUB/test_dir/SENTINEL_L1C/"

## Sentinel-HUb configuration

In [ ]:
from sentinelhub import SHConfig


INSTANCE_ID = '82f77578-eb9a-4909-9a28-30f5279ba6a3'  # In case you put instance ID into configuration file you can leave this unchanged

if INSTANCE_ID:
    config = SHConfig()
    config.instance_id = INSTANCE_ID
else:
    config = None

#### Imports 

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

### Import the Libraries

In [ ]:
# Please, install all the neccesary libs.
import datetime
import numpy as np

import matplotlib.pyplot as plt
import os 
import json 

In [ ]:
from sentinelhub import WmsRequest, WcsRequest, MimeType, CRS, BBox, DataCollection

In [ ]:
def plot_image(image, factor=1):
    """
    Utility function for plotting RGB images.
    """
    fig = plt.subplots(nrows=1, ncols=1, figsize=(15, 7))
    
    if np.issubdtype(image.dtype, np.floating):
        plt.imshow(np.minimum(image * factor, 1))
    else:
        plt.imshow(image)

### Setting parameters to rename the folder for campo1

In [ ]:
satellite='SentinelL1C'
campo1='campo1'

#### Setting area of interest

In [ ]:
talca_coords_wgs84 = [-71.817933, -35.452880, -71.807166, -35.446550] #TALCA coordinates 1st PART (Parte izquierda del terreno)
#FORMAT(LONGITUDE1, LATITUDE1, LONGITUDE2, LATITUDE2) From Google Earth they are lower left and upper rigth corners 

All requests require bounding box to be given as an instance of `sentinelhub.geometry.BBox` with corresponding Coordinate Reference System (`sentinelhub.geometry.CRS`). In our case it is in _WGS84_ and we can use the predefined _WGS84_ coordinate reference system from `sentinelhub.geometry.CRS`.

In [ ]:
talca_bbox = BBox(bbox=talca_coords_wgs84, crs=CRS.WGS84)

In [ ]:
wms_true_color_request = WmsRequest(
    data_collection=DataCollection.SENTINEL2_L1C,
    data_folder=rootPath,
    layer='TRUE-COLOR-S2-L1C',
    bbox=talca_bbox,
    #time='2017-12-10',
    time=('2016-01-01','2016-12-31'),
    #width=512,
    #height=856,
    width=600,
    #height=600,
    image_format=MimeType.TIFF,
    time_difference=datetime.timedelta(hours=2),
    config=config
)

In [ ]:
#Save data
%%time
wms_true_color_request.save_data()


CPU times: user 230 ms, sys: 33.4 ms, total: 264 ms
Wall time: 4.84 s


## PRUEBA DE NUMERO DE IMAGENES Q DESCARGA

In [ ]:
wms_bands_img_from_disk = wms_true_color_request .get_data()
print('These %d images were taken on the following dates:' % len(wms_bands_img_from_disk))
for index, date in enumerate(wms_true_color_request.get_dates()):
    print(' - image %d was taken on %s' % (index, date))

These 18 images were taken on the following dates:
 - image 0 was taken on 2016-01-05 14:52:57
 - image 1 was taken on 2016-02-04 14:42:56
 - image 2 was taken on 2016-03-05 14:51:55
 - image 3 was taken on 2016-04-04 14:41:37
 - image 4 was taken on 2016-05-04 14:41:36
 - image 5 was taken on 2016-05-24 14:51:40
 - image 6 was taken on 2016-06-13 14:50:55
 - image 7 was taken on 2016-07-03 14:48:53
 - image 8 was taken on 2016-07-23 14:49:33
 - image 9 was taken on 2016-08-12 14:49:22
 - image 10 was taken on 2016-09-01 14:49:26
 - image 11 was taken on 2016-09-21 14:49:46
 - image 12 was taken on 2016-10-11 14:49:22
 - image 13 was taken on 2016-10-31 14:48:10
 - image 14 was taken on 2016-11-10 14:52:12
 - image 15 was taken on 2016-11-30 14:52:15
 - image 16 was taken on 2016-12-10 14:45:55
 - image 17 was taken on 2016-12-20 14:51:31


In [ ]:
#Getting the list of all saved folders.
folders1=os.listdir(wms_true_color_request.data_folder)
print(folders1)

['9d9685871c34f94a77b149918ebccc44', 'e9409e738a426ca913bbc7afbfa149ac', 'd5cb79df1531c1794aa4c254feb81aa7', '69c079f1bc473bd05df94d77a3236e12', '9128f6370999bcadac2cf35bc790662a', '45da053f171d6a50b027c2740c1e47ab', 'd181fcd8e2bba6aef6f2717195aa3a92', '9e58f0687da232c0f7659e0d760d55fc', 'deaeaf81dde5deeded9d46445d9eaff5', '01224daefd8c890a7d97db2ec3979df2', '54b89c1ec3b8c2f880ca960cc90fec87', '4c19028f5c84020275a085c8ef59f0fe', '27f79fb928863a48e5f0c3c7680f822e', 'f51d17ef09d3559131d1d772a2052a15', 'ec2e73b61b60a4b45733e740e5f31f07', '8199563785ba6d932724137d20610fce', 'a8fc401338baa2b6f4084687d6bd128b', '97f346b080c7169e0b4d01b64013f974']


### Setting parameters to rename the folder

In [ ]:
satellite='SentinelL1C'
campo2='campo2'


#### Setting area of interest

In [ ]:
talca_coords_wgs84 = [-71.800228, -35.457577, -71.786453, -35.449821] #TALCA coordinates 2nd PART (Parte derecha del terreno)
#FORMAT(LONGITUDE1, LATITUDE1, LONGITUDE2, LATITUDE2) From Google Earth they are lower left and upper right corners 

All requests require bounding box to be given as an instance of `sentinelhub.geometry.BBox` with corresponding Coordinate Reference System (`sentinelhub.geometry.CRS`). In our case it is in _WGS84_ and we can use the predefined _WGS84_ coordinate reference system from `sentinelhub.geometry.CRS`.

In [ ]:
talca_bbox = BBox(bbox=talca_coords_wgs84, crs=CRS.WGS84)

In [ ]:
wms_true_color_request = WmsRequest(
    data_collection=DataCollection.SENTINEL2_L1C,
    data_folder=rootPath,
    layer='TRUE-COLOR-S2-L1C',
    bbox=talca_bbox,
    #time='2017-12-10',
    time=('2016-01-01','2016-12-31'),
    #width=512,
    #height=856,
    width=600,
    #height=600,
    image_format=MimeType.TIFF,
    time_difference=datetime.timedelta(hours=2),
    config=config
)

In [ ]:
#Save data
%%time
wms_true_color_request.save_data()


CPU times: user 230 ms, sys: 44.6 ms, total: 274 ms
Wall time: 2.56 s


## PRUEBA DE NUMERO DE IMAGENES Q DESCARGA

In [ ]:
wms_bands_img_from_disk = wms_true_color_request .get_data()
print('These %d images were taken on the following dates:' % len(wms_bands_img_from_disk))
for index, date in enumerate(wms_true_color_request.get_dates()):
    print(' - image %d was taken on %s' % (index, date))

These 18 images were taken on the following dates:
 - image 0 was taken on 2016-01-05 14:52:57
 - image 1 was taken on 2016-02-04 14:42:56
 - image 2 was taken on 2016-03-05 14:51:55
 - image 3 was taken on 2016-04-04 14:41:37
 - image 4 was taken on 2016-05-04 14:41:36
 - image 5 was taken on 2016-05-24 14:51:40
 - image 6 was taken on 2016-06-13 14:50:55
 - image 7 was taken on 2016-07-03 14:48:53
 - image 8 was taken on 2016-07-23 14:49:33
 - image 9 was taken on 2016-08-12 14:49:22
 - image 10 was taken on 2016-09-01 14:49:26
 - image 11 was taken on 2016-09-21 14:49:46
 - image 12 was taken on 2016-10-11 14:49:22
 - image 13 was taken on 2016-10-31 14:48:10
 - image 14 was taken on 2016-11-10 14:52:12
 - image 15 was taken on 2016-11-30 14:52:15
 - image 16 was taken on 2016-12-10 14:45:55
 - image 17 was taken on 2016-12-20 14:51:31


### Etiquetar las imágenes con el # de campo y fecha respectiva.

In [ ]:
#Getting the list of all saved folders.
folders2=os.listdir(wms_true_color_request.data_folder)

for i in folders1:
  for j in folders2:
    if i == j:
      folders2.remove(j)
print(folders1)
print(folders2)

['9d9685871c34f94a77b149918ebccc44', 'e9409e738a426ca913bbc7afbfa149ac', 'd5cb79df1531c1794aa4c254feb81aa7', '69c079f1bc473bd05df94d77a3236e12', '9128f6370999bcadac2cf35bc790662a', '45da053f171d6a50b027c2740c1e47ab', 'd181fcd8e2bba6aef6f2717195aa3a92', '9e58f0687da232c0f7659e0d760d55fc', 'deaeaf81dde5deeded9d46445d9eaff5', '01224daefd8c890a7d97db2ec3979df2', '54b89c1ec3b8c2f880ca960cc90fec87', '4c19028f5c84020275a085c8ef59f0fe', '27f79fb928863a48e5f0c3c7680f822e', 'f51d17ef09d3559131d1d772a2052a15', 'ec2e73b61b60a4b45733e740e5f31f07', '8199563785ba6d932724137d20610fce', 'a8fc401338baa2b6f4084687d6bd128b', '97f346b080c7169e0b4d01b64013f974']
['6f3e7136d1a62e7ac9bf00bac05a8f5d', '923c0ceea300c7c56821c032588608ea', 'd98989c62358f413291e97b22c4f829d', 'd80a4c5f0f0497013f2641ec0f1b5384', '24d1e75ebff01c3b36a1ee9836db70ab', 'b9e43ffeb8a23ece23033c1a4afabe08', '884a61543636e0481a980464dacffc36', '1e2d36d99346181c76fc9e84092ccf79', 'ac0f8456fd73405c6000d9eed4e41e05', '4b0d3cb4c48fbc58299f00eb3

In [ ]:
for folder in folders1:
  with open(rootPath+folder+"/request.json") as file: #Define the path to the folder and to the json file
    dic=json.load(file)
  dic1=dic.copy() #For the anotations
  url=dic1['url']  #key url where is the date
  date=[]
  date_str=""
  for i in range(len(url)):
      if url[i]=='T' and url[i+1]=='I' and url[i+2]=='M' and url[i+3]=='E':
          for j in range(10):
              date.append(url[i+5+j])
  date_str=date_str.join(date)
  date_str1=campo1+"_"+date_str
  image_path=rootPath+folder+"/response.tiff"
  new_name=rootPath+folder+"/"+date_str1+".tiff"
  os.rename(image_path,new_name)

  #Change the name of the folder
  folder_name=rootPath+folder
  new_folder_name=rootPath+campo1+"_"+satellite+"_"+date_str
  os.rename(folder_name,new_folder_name)

   


### Etiquetar las imágenes con el # de campo y fecha respectiva.

In [ ]:
for folder in folders2:
  with open(rootPath+folder+"/request.json") as file: #Define the path to the folder and to the json file
    dic=json.load(file)
  dic1=dic.copy() #For the anotations
  url=dic1['url']  #key url where is the date
  date=[]
  date_str=""
  for i in range(len(url)):
      if url[i]=='T' and url[i+1]=='I' and url[i+2]=='M' and url[i+3]=='E':
          for j in range(10):
              date.append(url[i+5+j])
  date_str=date_str.join(date)
  date_str1=campo2+"_"+date_str
  image_path=rootPath+folder+"/response.tiff"
  new_name=rootPath+folder+"/"+date_str1+".tiff"
  os.rename(image_path,new_name)

  #Change the name of the folder
  folder_name=rootPath+folder
  new_folder_name=rootPath+campo2+"_"+satellite+"_"+date_str
  os.rename(folder_name,new_folder_name)

   


In [ ]:
numero_carpetas=os.listdir(rootPath)
print(len(numero_carpetas))

36
